In [3]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
# import os

# os.chdir('drive/MyDrive/three-phase-fidelity-wip')
# print(os.getcwd())

Mounted at /content/drive
/content/drive/MyDrive/three-phase-fidelity-wip


In [2]:
# # # # !pip install -r requirements.txt
# !pip install ipython==7.31.1
# !pip install importlib-metadata==4.10.0
# !pip install acv-exp
# !pip install hyperopt==0.27
# # # ## !pip freeze > requirements.txt

     |████████████████████████████████| 792 kB 7.3 MB/s 
     |████████████████████████████████| 380 kB 53.7 MB/s 
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.28 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.31.1 which is incompatible.


  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.11.2
    Uninstalling importlib-metadata-4.11.2:
      Successfully uninstalled importlib-metadata-4.11.2
     |████████████████████████████████| 6.8 MB 8.0 MB/s 
     |████████████████████████████████| 405 kB 41.8 MB/s 
     |████████████████████████████████| 9.9 MB 28.9 MB/s 
     |████████████████████████████████| 22.3 MB 1.2 MB/s 
     |████████████████████████████████| 111 kB 67.5 MB/s 
     |████████████████████████████████| 4.3 MB 46.4 MB/s 
     |████████████████████████████████| 181 kB 70.9 MB/s 
     |████████████████████████████████| 164 kB 66.5 MB/s 
     |████████████████████████████████| 76 kB 5.3 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 130 kB 64.9 MB/s 
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=0ac9099d7099f39832bb025a34b636dc273b3673787f600f1cfbabbe3d06af2d
  Stored in di

ERROR: Could not find a version that satisfies the requirement hyperopt==0.27 (from versions: 0.0.1, 0.0.2, 0.1, 0.1.1, 0.1.2, 0.2, 0.2.1, 0.2.2, 0.2.3, 0.2.4, 0.2.5, 0.2.6, 0.2.7)
ERROR: No matching distribution found for hyperopt==0.27


In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import f1_score, classification_report, roc_auc_score, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import KFold
import sklearn

import os
import joblib

import warnings
warnings.filterwarnings('ignore')

from acv_explainers import ACXplainer

import random

from tqdm import tqdm_notebook

from hyperopt import fmin, tpe, hp, Trials
from hyperopt.pyll import scope

In [4]:
# path to project folder
# please change to your own
PATH = os.getcwd()

dataset = "iris"
cls_method = "decision_tree"

classification = True

random_state = 22
exp_iter = 10

save_to = "%s/%s/" % (PATH, dataset)
dataset_folder = "%s/datasets/" % (save_to)
final_folder = "%s/%s/" % (save_to, cls_method)

#Get datasets
X_train = pd.read_csv(dataset_folder+dataset+"_Xtrain.csv", index_col=False, sep = ";")
y_train = pd.read_csv(dataset_folder+dataset+"_Ytrain.csv", index_col=False, sep = ";")
test_x = pd.read_csv(final_folder+"test_sample.csv", index_col=False, sep = ";").values
results = pd.read_csv(os.path.join(final_folder,"results.csv"), index_col=False, sep = ";")
actual = results["Actual"].values

feat_list = [each.replace(' ','_') for each in X_train.columns]

In [5]:
cls = joblib.load(save_to+cls_method+"/cls.joblib")
scaler = joblib.load(save_to+"/scaler.joblib")

In [6]:
Y_pred = cls.predict(X_train)
test_pred = cls.predict(test_x)
kf = KFold(n_splits=5, shuffle = True, random_state=random_state)

space = {"n_estimators": scope.int(hp.quniform('n_estimators', 1, 100, q=1)),
        "max_depth": scope.int(hp.quniform('max_depth', 1, 100, q=1))}

trials = Trials()

if classification:
    
    def acv_classifier_optimisation(args, random_state = random_state, cv = kf, X = X_train.values, y = Y_pred):
        score = []
        for train_index, test_index in kf.split(X):

            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            estimator = ACXplainer(classifier = True, n_estimators = args["n_estimators"], max_depth = args['max_depth'])
            estimator.fit(X_train, y_train)

            score.append(roc_auc_score(y_test, estimator.predict(X_test)))
        
        score = np.mean(score)
        return -score
    
    best = fmin(acv_classifier_optimisation, space = space, algo=tpe.suggest, max_evals = 50, trials=trials, rstate = np.random.RandomState(random_state))
    explainer = ACXplainer(classifier = True, n_estimators = int(best['n_estimators']), max_depth = int(best['max_depth']))
    explainer.fit(X_train, Y_pred)
    
    print(classification_report(actual, explainer.predict(test_x)))

else:
    
    def acv_regression_optimisation(args, random_state = random_state, cv = kf, X = X_train.values, y = Y_pred):
        score = []
        
        for train_index, test_index in kf.split(X):

            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y[train_index], y[test_index]

            estimator = ACXplainer(classifier = False, n_estimators = args["n_estimators"], max_depth = args['max_depth'])
            estimator.fit(X_train, y_train)

            score.append(mean_absolute_percentage_error(y_test, estimator.predict(X_test)))
            
        score = np.mean(score)
        return score
    best = fmin(acv_regression_optimisation, space = space, algo=tpe.suggest, max_evals = 50, trials=trials, rstate = np.random.RandomState(random_state))
    explainer = ACXplainer(classifier = False, n_estimators = int(best['n_estimators']), max_depth = int(best['max_depth']))
    explainer.fit(X_train, Y_pred)

    print("MAPE:", mean_absolute_percentage_error(actual, explainer.predict(test_x)))
    print("R-Squared:", r2_score(actual, explainer.predict(test_x)))

joblib.dump(explainer, save_to+cls_method+"/acv_explainer.joblib")

100%|██████████| 50/50 [00:03<00:00, 15.00it/s, best loss: -1.0]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      1.00      1.00        13

    accuracy                           1.00        26
   macro avg       1.00      1.00      1.00        26
weighted avg       1.00      1.00      1.00        26



['/content/drive/MyDrive/three-phase-fidelity-wip/iris/decision_tree/acv_explainer.joblib']